# Phénomène de Runge

On considère la fonction $f(x) = \displaystyle \frac{1}{(1 + 25x^2)}$.

In [ ]:
def f(x):
    return 1/(1+25*x*x)

## Interpolation utilisant des points équidistants

On interpole la fonction $f(x)$ en utilisant des points équidistants avec la formulation de Newton grâce à la fonction du module  `scipy` : [interpolate.KroghInterpolator](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.KroghInterpolator.html#scipy.interpolate.KroghInterpolator).

In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy import interpolate

xmin = -1.0
xmax = 1.0

# maximal degree of Newton interpolating polynomial
n_max = 30

# array of degree
arr_n = np.arange(1, n_max)

# compute for each degree xi, yi and pn
xi_equi = []; yi_equi = []; pn_equi = []
for i, ni in enumerate(arr_n):
    xi_equi.append(np.linspace(xmin, xmax, ni+1))
    yi_equi.append(f(xi_equi[i]))
    pn_equi.append(interpolate.KroghInterpolator(xi_equi[i], yi_equi[i]))
    
# Create figure
fig = go.Figure()

# x use to display f and pn
xplot = np.linspace(xmin, xmax, 500)

# add f(x) plot
fig.add_trace(go.Scatter(visible=True, x=xplot, y=f(xplot), name="f(x)"))

# add yi and pn(x) invisible plots
for i, ni in enumerate(arr_n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=pn_equi[i](xplot), name=f"p{ni}(x)", marker=dict(color='forestgreen')))
    fig.add_trace(go.Scatter(visible=False, x=xi_equi[i], y=yi_equi[i], mode='markers', name="interpolating points", marker=dict(color='forestgreen')))

# Make plot visible for n=1
fig.data[1].visible = True
fig.data[2].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(arr_n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(currentvalue={"prefix": "nb points: "}, steps=steps)]

fig.update_layout(sliders=sliders)
fig.update_yaxes(range=[-.25, 1.25])

fig.show()

Lorsque le nombre de points d'interpolation augmentent, on remarqe que le polynôme se met à osciller avec une amplitude de plus en plus grande. C'est le **phénomène de Runge**.

## Interpolation utilisant les points de Tchebychef

In [ ]:
def cheb_points(xmin, xmax, n):
    x = np.zeros(n+1)
    for i in range(n+1):
        x[i] = (xmax+xmin)/2 + ((xmax-xmin)/2) * np.cos(((2*i+1)*np.pi)/(2*n + 2))
    return x

In [ ]:
from plotly.subplots import make_subplots

# compute for each degree xi, yi and pn
xi_cheb = []; yi_cheb = []; pn_cheb = []
for i, ni in enumerate(arr_n):
    xi_cheb.append(cheb_points(xmin, xmax, ni))
    yi_cheb.append(f(xi_cheb[i]))
    pn_cheb.append(interpolate.KroghInterpolator(xi_cheb[i], yi_cheb[i]))

fig = make_subplots(rows=1, cols=2, subplot_titles=("Points équidistants ", "Points de Tchebychef"))

# add f(x) plot
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name="f(x)", legendgroup="f", marker=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name="f(x)", legendgroup="f", showlegend=False, marker=dict(color='blue')), row=1, col=2)

# add yi and pn(x) invisible plots
for i, ni in enumerate(arr_n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=pn_equi[i](xplot), name=f"p{ni}(x)", 
                            marker=dict(color='forestgreen')), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=xi_equi[i], y=yi_equi[i], mode='markers', showlegend=False,
                             marker=dict(color='forestgreen')), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=pn_cheb[i](xplot), showlegend=False,
                              marker=dict(color='forestgreen')), row=1, col=2)
    fig.add_trace(go.Scatter(visible=False, x=xi_cheb[i], y=yi_cheb[i], mode='markers', showlegend=False,
                             marker=dict(color='forestgreen')), row=1, col=2)
    
# Make plot visible for n=1
fig.data[2].visible = True
fig.data[3].visible = True
fig.data[4].visible = True
fig.data[5].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(arr_n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==1 or el==4*i+2 or el==4*i+3 or el==4*i+4 or el==4*i+5 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(currentvalue={"prefix": "nb points: "}, steps=steps)]

fig.update_layout(sliders=sliders)
fig.update_yaxes(range=[-.25, 1.25])
fig.show()

On remarque une nette amélioration de l'interpolation en utilisant les points de Tchebychef.